# 1.1 Searching for $H\rightarrow b\bar{b}$

One of the primary physics objectives of the Large Hadron Collider is to study the Higgs Boson and understand the process with which particles acquire mass. The Higgs Boson's decays channels along with the respective branching ratios are shown in the figure below. 

<img src="https://physicsforme.files.wordpress.com/2012/07/higgs_decay.jpg" width="400" />



In 2012 the Higgs Boson was discovered in the $H\rightarrow\gamma\gamma$ and $H\rightarrow ZZ \rightarrow 4l$. The plot below shows a clear excess of signal events over background around 125 GeV in the $H\rightarrow ZZ \rightarrow 4l$ 'golden channel'. 

<img src="https://3c1703fe8d.site.internapcdn.net/newman/gfx/news/hires/2017/newatlasprec.png" width="400" />


But if the Higgs Boson predominantly decays to a pair of $b$-quarks, why wasn't it discovered in the $H\rightarrow b\bar{b}$ channel? 

The discovery channels leave a clean signature in the detector that is easy to isolate making searches in the $H\rightarrow\gamma\gamma$ or $H\rightarrow ZZ \rightarrow 4l$ favourable. However, $H\rightarrow b\bar{b}$ searches aren't so straightforward!

Here's why:

## Large background 

Firstly, the $H\rightarrow b\bar{b}$ process at the LHC has enourmous amount of background with final states that are the same as the signal i.e. two $b$-quarks. The figure below shows the cross sections (likelihood) of a Higgs event and compared to a bb event at the LHC. In the LHC current operational energy, only around 1 in a trillion proton-proton collisions create a Higgs boson. 


<img src="../docs/images/lhc_cross_sections.png" width="400" />

Hence, the signal can get drowned out in with the signal-like background. 

## Jet Production

Secondly, precisely measuring and distinguishing $b$-quarks from say $c$-quarks is also challenging. When a pair of $b$-quarks is created in the ATLAS detector they go through a process known as hadronisation and form [jets](https://www.youtube.com/watch?v=FMH3T05G\_to). The jets must be identified as originating from b-quarks (b-jets) by a process known as $b$-tagging. This extra level of complexity along with the large background makes searches for $H\rightarrow b\bar{b}$ challenging. 

## 1.2 One - Lepton Channel

The process we will be searching for is shown in the Feynman Diagram below. A Higgs Boson is radiated off a $W^\pm$ boson which subsequently decays to a pair of _b_-quarks. The $W^\pm$ boson then goes on to decay into a lepton and a corresponding neutrino. 

<img src="images/one-lepton.png" width="350" />



The final state products of a 1-lepton channel $H\rightarrow b\bar{b}$ process are:
   * A Neutrino
   * A charged Lepton (e u)
   * 2 _b_-jets


## 1.4 Separating Signal from Background


We separate ATLAS events using *Kinematic* (momentum, energy, mass etc) and Topological parameters (spatial separation between particles). A list of the event variables that we will be using in this exercise is shown below: 



| Variable        | Description           | Label  |
| ------------- |:-------------:| -----:|
|$\Delta R(b_1b_2)$      | Angle between the two *b*-tagged jets | dRBB |
| $p_T^V$      | Missing Transverse Momentum      |   pTV |
| $m_{top}$ | Reconstructed Top Quark Mass      |    Mtop |
| $E^{Miss}_{T}$ | Missing Transverse Energy     |    MET |


                    Table 1: Kinematic and topological paramaeters used to identify events. 



# 2.0 Finding signal in the data
In particle physics, we want to try and find some evidence of a signal process, such as the production of a Higgs boson, which then decays to 2 b-jets. We do this by measuring the final particles, the b-jets, and then adding up their properties to find the properties of the Higgs. By doing this, we can find properties such as the Higgs mass. In the first example below, we show an ideal case, where we have a lot of Higgs bosons produced, and no background.


In [ ]:
from pathlib import Path
if not Path("VHbb_data_2jet.csv").exists():
    # Download dataset, and a custum library from github
    !wget https://raw.githubusercontent.com/nikitapond/in2HEP/StudentWeek/data-v2/VHbb_data_2jet.csv
if not Path("ucl_masterclass.py").exists():
    !wget https://raw.githubusercontent.com/nikitapond/in2HEP/StudentWeek/notebooks/ucl_masterclass.py
# Importing required libraries
import pandas as pd
import numpy as np
from copy import deepcopy
from ucl_masterclass import *
import matplotlib as mpl

## 2.1.1 An Ideal Case

In [ ]:
def generate_dummy_signal(n):
    '''
    Generates a dummy Higgs signal with a mean of 125 and a standard deviation of 1.
    '''
    return 125 + np.random.randn(n)*5

def make_hist(data, title):
    '''
    Makes a histogram of the data.
    '''
    plt.hist(data, bins=np.linspace(50, 200, 50));
    plt.xlabel("Mass [GeV]")
    plt.ylabel("Number of Events")
    plt.title(title)

make_hist(generate_dummy_signal(20000), "Dummy signal data")

## 2.1.2 What about background

In the above example, we can clearly see a peak at 125 GeV, highlighting a particle that exists with this mass. But this is an ideal case, with a lot of signal, and no background. What happens if we change these two factors?



In [ ]:
def generate_dummy_background(n):

    exp_bkg = np.random.exponential(100, n)
    gaus_bkg_1 = np.random.randn(n)*30 + 100
    gaus_bkg_2 = np.random.randn(n)*30 + 200
    return np.concatenate([exp_bkg, gaus_bkg_1, gaus_bkg_2])

# Generate a dummy background, caused by several other processes
dummy_back = generate_dummy_background(20000)
# Generate a dummy signal, caused by the Higgs boson, with high statistics
dummy_signal = generate_dummy_signal(20000)
# What we actually measure is all the signal and background
dummy_measured = np.concatenate([dummy_back, dummy_signal])
make_hist(dummy_measured, "Dummy measured data")

## 2.1.3 Lowering the signal
The actual rate at which Higgs bosons is produced is signficantly smaller than the background. Lets see how that effects these plots

In [ ]:
# Generate a dummy background, caused by several other processes
dummy_back = generate_dummy_background(10000)
# Generate a dummy signal, caused by the Higgs boson, with low statistics
dummy_signal = generate_dummy_signal(250)
# What we actually measure is all the signal and background
dummy_measured = np.concatenate([dummy_back, dummy_signal])

make_hist(dummy_measured, "Dummy measured data")

The above plot will look different each time you run it, due to the random numbers utilised. This is very similar to what actually happens in the ATLAS detector, with random fluctations in data occuring. Sometimes, you might get lucky and see a clear peak around 125 GeV, but a lot of the time you wont. Even worse, you might see random peaks elsewhere! So, how can we find our Higgs boson when there's so much other stuff going on? We can apply cuts!

## 2.1.4 Ideal cuts
In this ideal example, we shall apply 'cuts' which just reduce the amount of background and signal we have, but at ideal rates

In [ ]:
bgk_factor = 0.5
signal_factor = 0.9
num_cuts = 4
bkg_reduced = 1.0
sig_reduced = 1.0

dummy_back = generate_dummy_background(10000)
dummy_signal = generate_dummy_signal(250)

plt.title("Measured data after cuts")
plt.xlabel("Mass [GeV]")
plt.ylabel("Number of Events")
plt.hist(np.concatenate([dummy_back, dummy_signal]), bins=np.linspace(50, 200, 50), label="No cuts");
for i in range(num_cuts):
    bkg_reduced *= bgk_factor
    sig_reduced *= signal_factor
    dummy_measured = np.concatenate([dummy_back[:int(len(dummy_back)*bkg_reduced)], dummy_signal[:int(len(dummy_signal)*sig_reduced)]])
    plt.hist(dummy_measured, bins=np.linspace(50, 200, 50), label=f"Cut {i+1}");
plt.legend()

In this final plot, we can see that after applying a series of cuts to our signal and background, we eventually start to see our peak more and more clearly. Once we can see a peak clearly above the background noise, we can use it to declare that we've found a Higgs.

When it actually comes to declaring we've discovered a particle, we need to ensure that the peak we see is larger than the random noise fluctuations we see in the background. If we don't do this check, and say every peak we see is a new particle, we'd be discovering new particles every week! We aim to find a peak which is statistically significant compared to the background, this means we calculate the probability that a peak is down to random noise. To discover a particle, we require the probability of the peak being caused by random noise to be approximatly 1 in a billion!

## 2.2 A more realistic example
In the above code, we generated some dummy data, with a clear peak from the Higgs boson decay, and a few different possible background processes. When applying cuts, we just imaged we were able to apply a cut that reduced the background by 50%, while only reducing the signal by 10%. We also only generated a single variable, the sum of the mass of the two b-jets. In real data, each point represents an event in the ATLAS detector. But how do we know if the cuts we're applying are working? The way we do this is to instead work with simulations, and compare these to what we see in the data. These are called Monte-Carlo (MC) simulations, as they involve random numbers, as they simulate quantum processes which have an intrinstic random nature. When we generate MC, we know what processes we have generated, and so we can plot all the different background and signal events with different colours, to make it clear what each processes is contributing. Below, lets look at the distributions for some of our variables:

In [ ]:
# Load data into a pandas data frame
df = pd.read_csv('VHbb_data_2jet.csv')
df_original = deepcopy(df)

# The plot_variable takes two arguments. First is the data frame used to plot the distributions and
# second is the variable in question.
plot_variable(df,'mBB') # Draw the mBB distribution
plot_variable(df,'pTB1') # Draw the pTB1 distribution
plot_variable(df,'Mtop') # Draw the pTV distribution

### 2.2.1 What is Sensitivity?
Sensitivity is the quantative measure of how much our peak sticks up above the background. It has a fancy calculation that we don't need to get into, but the rough idea is this:

- Take our histogram of signal and background
- For each bin, calculate a per bin sensitivity. The per bin sensitivity increases when we have more signal in that bin.
- Add up the per-bin sensitivity

In the below cell, we calculate the sensitivity on the variable, this is the mass we get by adding the two b-jets, which for signal represents the mass of the Higgs boson.


In [ ]:
# The code below plots the mBB distribution before any selection is applied
plot_variable(df_original,'mBB')
# Calculate and output the sensitivity based up the original mBB distribution prior to any selection
# The sensitivity is calculated using the profile likelihood ratio test and Asimov approach
print("Sensitivity achieved before cuts ",sensitivity_cut_based(df_original))

## 2.3 $H\rightarrow b\bar{b}$ via Sequential Cuts

In this section we are going to be applying a set of selection cuts on the kinematic and topological paramaters to select signal events and reject background events. We will be applying cuts on all variables other than $m_{bb}$ and see what effect the cut has on the $m_{bb}$ distribution histogram. 

The $m_{bb}$ is the reconstructed mass of the two jets, if we assume both to be caused by $b$ quarks. For signal events, we expect this value to be very close to the Higgs mass of 125 GeV. We show this variable plot below




In [ ]:
# The data is loaded into a pandas data frame
df = pd.read_csv('VHbb_data_2jet.csv')

# Later on, we shall be changing the data held in the 'df'
# variable above. We make a copy here of the original to
# compare results against
df_original = deepcopy(df)

# here we plot 'mBB' - the mass of the jets if we assume they come from
# b-jets. 
plot_variable(df,'mBB')

This plot shows a histogram of the number of events in each 'bin', where 'bin' refers to a range in $m_{bb}$. Signal events (from Higgs) are shown as red blocks. A line representing the concentration of signal events is also included in red. This plot shows, as expected, the highest distribution of signal events to have a mass of ~125 GeV. For the remainder of this notebook, we shall use this variable plot to highlight the effect our other cuts have.

To evaluate the cuts we will compare the number of signal and background events in each bin to calculate the _signal sensitivity_. Our goal is to apply cuts that isolate as much of the signal as possible and maximise the _signal sensitivity_. 


# 2.4 Finding Cuts

**In this section we will be using the simulated data to determine what cuts must be applied on the kinematic and topoligical parameters to identify the signal region.**


To do this we will use the plot_variable() function to visualise how a particular kinematic or topological event variable is distributed for the various process involved. We wll use this distribution to determine the appropriate cuts. Let's use an example to illustrate this concept.

The code below plots the distribution of the reconstructed top mass $M_{top}$. The signal (labelled $VH \rightarrow bb$ in the legend) is shown in red on the histogram. For clarity a red solid line has also been plotted to help identify the sigal region. 

In [ ]:
# The plot_variable takes two arguments. First is the data frame used to plot the distributions and
# second is the variable in question. 

plot_variable(df,'MET')

### Exercise 2.4.1 Use the plot variable function to plot another variable (like Mtop) which is the reconstructed mass of the top quark.

In [ ]:
# make your plot here!


### Exercise 2.4.1 Plot the event variables and determine the cuts that will isolate the signal region. Note these cuts down in your notebook.


An example is shown in the cells below where it was identified that by requiring that Mtop > 225 GeV we eliminate a large amount of the $t\bar{t}$ and single top background. 

By changing the variable plotted determine your own list of cuts. 

In [ ]:
# Reload data
df = pd.read_csv('VHbb_data_2jet.csv')

# Apply cut
df = df.loc[df["MET"]/1000 > 50] # Select only events with MET > 50 GeV

# The code below plots the mBB distribution after the cut is applied
plot_variable(df_original,'mBB')
print("Sensitivity achieved before cuts ",sensitivity_cut_based(df_original))

plot_variable(df,'mBB')

print("Sensitivity achieved after cuts ",sensitivity_cut_based(df))

As shown in the above plots of the $m_{bb}$ distribution, once the cut is applied we eliminate a large amount of $t\bar{t}$ background.

### Determine a set of additional cuts that will help isolate the signal. Each time you optimise a new cut value, note down the new sensitivity. After you have optimised all the cuts, plot the sensitivity increase as each new cut is applied. Code to create plots in python is provided below. 



In [ ]:
# Reload data
df = pd.read_csv('VHbb_data_2jet.csv')

# Apply a cut here!
# =================



# The code below plots the mBB distribution after the cut is applied
plot_variable(df_original,'mBB')
print("Sensitivity achieved before cuts ",sensitivity_cut_based(df_original))

plot_variable(df,'mBB')

print("Sensitivity achieved after cuts ",sensitivity_cut_based(df))

In [ ]:
## Plotting sensitivity graph

# Add the sensitivities you noted down to the list
sensitivities = [1.0, 1.1, 1.2, 1.3]

# Add variable names to the list below. Make sure you keep the same order
variable_names = ['dRBB', 'MET', 'Mtop', 'pTV']

num_variables_used = len(sensitivities)



fig = plt.figure()
fig.set_size_inches(8.5,7)
x = list(np.arange(0,num_variables_used,1))
plt.bar(x,sensitivities)

plt.xticks(x, variable_names);

plt.xlabel('Variables')
plt.ylabel('Sensitivity')


# Once you have finalised your plots you can export a .png file by uncommenting the code below
#plt.savefig('cut_based_variable_comparison.png')

### Exercise 2.4.3 Cutting multiple variables

So far, we have only asked you to cut on 1 variable at a time. Now, we shall cut on two variables



In [ ]:
# Reload data
df = pd.read_csv('VHbb_data_2jet.csv')

# The code below plots the mBB distribution after the cut is applied
plot_variable(df_original,'mBB')
print("Sensitivity achieved before cuts ",sensitivity_cut_based(df_original))


# Apply first cut
df = df.loc[df["Mtop"] / 1e3 > 250] # divide by 1e3 to convert from MeV to GeV
print("Sensitivity achieved after first cut ",sensitivity_cut_based(df))


# Apply second cut
df = df.loc[df["dRBB"] > 0.5] 
print("Sensitivity achieved after second cut ",sensitivity_cut_based(df))

plot_variable(df,'mBB')

print("Sensitivity achieved after cuts ",sensitivity_cut_based(df))

### Why does applying 2 cuts sometimes decrease sensitivity

You may have noticed that some cuts work when we apply them to a single variable, but once we cut two variables at once we sometimes decrease the sensitivity. 
To see why this happens, we below show how the plot for the variable 'dRBB' changes as we apply cuts on other variables first,

In [ ]:
# Reload data
df = pd.read_csv('VHbb_data_2jet.csv')


# The code below plots the mBB distribution after the cut is applied
print("Sensitivity achieved before cuts ",sensitivity_cut_based(df_original))
plot_variable(df,'dRBB')

# Apply first cut
df = df.loc[df["Mtop"] / 1e3 > 250] # divide by 1e3 to convert from MeV to GeV
print("Sensitivity achieved after first cut ",sensitivity_cut_based(df))
plot_variable(df,'dRBB')

# Apply second cut
df = df.loc[df["dRBB"] > 0.5] 
print("Sensitivity achieved after second cut ",sensitivity_cut_based(df))
plot_variable(df,'dRBB')


Above, we first plot the variable dRBB when no cut has been applied. We then plot the same variable after cutting on Mtop. We see that this drastically changes the distribution of dRBB, as shown in the second plot. Now, the point at which we cut to optimally increase the sensitivity may have moved, as the distribution has now changed.

Ideally, we need a method which allows us to place multiple cuts on different variables at once, taking into account how these distributions change as those cuts are applied. To do this in a hand written algorithm would be challenging. Instead, tomorrow we shall introduce a neural network approach that aims to select signal events from our dataset.

### Summary

In this tutorial you developed a cut-based analysis on Monte Carlo generated ATLAS data to search for the Higgs Boson's decay to a pair of _b_-quarks. This is a tried and tested approach that is much loved by physicists as one can theoretically motivate the cuts to create a signal region. But can we improve on this? In the next tutorial we will be using Machine Learning to push the boundaries of our analysis and try to obtain sensitivity gains. 